# Como construir um sistema de recomendação para dados de compra (passo a passo)
* Descrição: uma documentação sobre a construção de modelos de filtragem colaborativa para recomendar produtos aos clientes
* Link: https://medium.com/datadriveninvestor/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
* Autor: Moorissa Tjokro

## Declaração do problema
Neste desafio de dados, estamos construindo modelos de filtragem colaborativa para recomendar itens de produtos. As etapas a seguir têm como objetivo recomendar aos usuários seus 10 principais itens para colocar na cesta. A saída final será um arquivo csv na pasta `output` e uma função que procura uma lista de recomendações com base em um usuário especificado:
* Entrada: usuário - ID do cliente
* Retorna: lista classificada de itens (IDs de produto), que o usuário mais provavelmente deseja colocar em sua "cesta" (vazia)

## 1. Módulos de importação
* `pandas` e` numpy` para manipulação de dados
* `turicreate` para realizar a seleção e avaliação do modelo
* `sklearn` para dividir os dados no conjunto de treino e teste

In [17]:
%load_ext autoreload
%autoreload 2
!pip install turicreate

import pandas as pd
import numpy as np
import time
import turicreate as tc
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

#import sys
#sys.path.append("..")
#!pip install scripts
#import scripts.data_layer as data_layer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Carregar dados
Dois conjuntos de dados são usados ​​neste exercício, que podem ser encontrados na pasta `data`:
* `recommend_1.csv` consistindo em uma lista de 1000 IDs de cliente para recomendar como saída
* `trx_data.csv` consistindo em transações do usuário

O formato é o seguinte.

In [18]:
customers = pd.read_csv('/content/recommend_1.csv')
transactions = pd.read_csv('/content/trx_data.csv')

In [19]:
print(customers.shape)

customers.head()

(1000, 1)


,customerId
0,1553
1,20400
2,19750
3,6334
4,27773


In [20]:
print(transactions.shape)
transactions.head()

(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


## 3. Preparação de dados
* Nosso objetivo aqui é dividir cada lista de itens na coluna `produtos` em linhas e contar o número de produtos comprados por um usuário

In [21]:
# example 1: split product items
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head().set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0
2,2,111.0,107.0,29.0,11.0,11.0,11.0,33.0,23.0,NaN,NaN
3,3,164.0,227.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


### 3.1. Crie dados com usuário, item e campo de destino
* Esta tabela será uma entrada para nossa modelagem posterior
    * Nesse caso, nosso usuário é `customerId`,` productId` e `purchase_count`

In [32]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.29 minutes


In [34]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


### 3.2. Criar dummy
* Dummy para marcar se um cliente comprou aquele item ou não.
* Se alguém compra um item, `purchase_dummy` é marcado como 1
* Por que criar um dummy em vez de normalizá-lo?
    * Normalizar a contagem de compras, digamos por cada usuário, não funcionaria porque os clientes podem ter frequências de compra diferentes e não têm o mesmo gosto
    * No entanto, podemos normalizar os itens por frequência de compra para todos os usuários, o que é feito na seção 3.3. abaixo de.

In [19]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [64]:
data_dummy = create_data_dummy(data)

,customerId,productId,purchase_count,purchase_dummy
0,0,1,2,1
1,0,13,1,1
2,0,19,3,1
3,0,20,1,1
4,0,31,2,1
...,...,...,...,...
133580,28596,211,3,1
133581,28596,255,1,1
133582,28598,212,1,1
133583,28604,282,1,1


### 3.3. Normalizar os valores dos itens entre os usuários
* Para fazer isso, normalizamos a frequência de compra de cada item entre os usuários, primeiro criando uma matriz de item de usuário da seguinte maneira

In [21]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
(df_matrix.shape)

(24429, 300)

In [23]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.142857,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# create a table for input to the modeling

d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm

,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333
...,...,...,...
7327308,26542,299,0.000000
7327378,26633,299,0.000000
7327554,26873,299,0.000000
7327649,26998,299,0.000000


#### Defina uma função para normalizar dados




In [67]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()

* Podemos normalizar o histórico de compras, de 0 a 1 (com 1 sendo o maior número de compras para um item e 0 sendo 0 contagem de compras para aquele item).

## 4. Divida o conjunto de dados em treino e teste
* Dividir os dados em conjuntos de treinamento e teste é uma parte importante da avaliação da modelagem preditiva, neste caso, um modelo de filtragem colaborativa. Normalmente, usamos uma parte maior dos dados para treinamento e uma parte menor para teste.
* Usamos a proporção 80:20 para o tamanho do nosso conjunto de treino e teste.
* Nossa parte de treinamento será usada para desenvolver um modelo preditivo, enquanto a outra para avaliar o desempenho do modelo.
* Agora que temos três conjuntos de dados com contagens de compra, compras dummy e contagens de compra normalizadas, gostaríamos de dividir cada um.

In [26]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [27]:
# Using turicreate library, we convert dataframe to SFrame - this will be useful in the modeling part

train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [28]:
train_data

customerId,productId,purchase_count
11593,2,1
6730,198,2
2562,33,1
16231,239,1
20040,73,1
23274,15,1
12971,43,1
7223,13,2
247,85,1
7608,2,2


In [29]:
test_data

customerId,productId,purchase_count
23910,148,1
4999,54,1
26644,212,1
7938,14,1
111,57,1
8075,2,2
1870,175,1
3938,205,1
10026,255,1
3881,49,1


#### Define a função `split_data` para dividir os dados de treino e teste

In [30]:
# We can define a function for this step as follows

def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [31]:
# lets try with both dummy table and scaled/normalized purchase table

train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

## 5. Modelo básico
Antes de executar uma abordagem mais complicada, como filtragem colaborativa, gostaríamos de usar um modelo básico para comparar e avaliar modelos. Esse modelo normalmente usa uma abordagem muito simples, as técnicas usadas além dessa abordagem devem ser escolhidas se mostrarem precisão e complexidade relativamente melhores.

### 5.1. Usando um modelo de popularidade como modelo básico
* O modelo de popularidade usa os itens mais populares para recomendação. Esses itens são produtos com o maior número de vendas entre os clientes.
* Usamos a biblioteca `turicreate` para executar e avaliar os modelos de filtragem básico e colaborativo abaixo
* Os dados de treinamento são usados ​​para a seleção do modelo

#### Usando contagens de compra

In [32]:
# variables to define field names
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [33]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23277 users and 300 items.

Data prepared in: 0.151261s

106868 observations to process; with 300 unique items.

In [34]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 101020

recommendations finished on 2000/62483 queries. users per second: 99339.4

recommendations finished on 3000/62483 queries. users per second: 103573

recommendations finished on 4000/62483 queries. users per second: 107026

recommendations finished on 5000/62483 queries. users per second: 109426

...

recommendations finished on 59000/62483 queries. users per second: 101584

recommendations finished on 60000/62483 queries. users per second: 101647

recommendations finished on 61000/62483 queries. users per second: 101498

recommendations finished on 62000/62483 queries. users per second: 101466

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.3214285714285716 |  1   |
|     0      |     37    | 3.0727969348659006 |  2   |
|     0      |    248    | 3.0681818181818183 |  3   |
|     0      |     0     | 2.994344957587182  |  4   |
|     0      |     34    | 2.9551020408163264 |  5   |
|     0      |     3     | 2.8305439330543933 |  6   |
|     0      |    110    | 2.691860465116279  |  7   |
|     0      |     27    | 2.6850393700787403 |  8   |
|     0      |    245    | 2.6785714285714284 |  9   |
|     0      |     10    | 2.6564417177914113 |  10  |
|     1      |    132    | 3.3214285714285716 |  1   |
|     1      |     37    | 3.0727969348659006 |  2   |
|     1      |    248    | 3.0681818181818183 |  3   |
|     1      |     0     | 2.994344957587182  |  4   |
|     1      |     34    | 2.9551020408163264 |  5   |
|     1   

#### Definir uma função `model` para a seleção do modelo

In [35]:
# Since turicreate is very accessible library, we can define a model selection function as below

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [36]:
# variables to define field names
# constant variables include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

#### Usando dummy

In [37]:
# these variables will change accordingly
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23299 users and 300 items.

Data prepared in: 0.150667s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 81752.8

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     23    |  1.0  |  1   |
|    1553    |     4     |  1.0  |  2   |
|    1553    |    258    |  1.0  |  3   |
|    1553    |     35    |  1.0  |  4   |
|    1553    |    173    |  1.0  |  5   |
|    1553    |     5     |  1.0  |  6   |
|    1553    |     96    |  1.0  |  7   |
|    1553    |     36    |  1.0  |  8   |
|    1553    |     19    |  1.0  |  9   |
|    1553    |     43    |  1.0  |  10  |
|   20400    |     23    |  1.0  |  1   |
|   20400    |     4     |  1.0  |  2   |
|   20400    |    258    |  1.0  |  3   |
|   20400    |     35    |  1.0  |  4   |
|   20400    |    173    |  1.0  |  5   |
|   20400    |     5     |  1.0  |  6   |
|   20400    |     96    |  1.0  |  7   |
|   20400    |     36    |  1.0  |  8   |
|   20400    |     19    |  1.0  |  9   |
|   20400    |     43    |  1.0  |  10  |
|   19750    |     23    |  1.0  |

#### Usando contagem normalizada

In [38]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23350 users and 300 items.

Data prepared in: 0.153352s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 96609

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7615894039735099 |  1   |
|    1553    |    247    | 0.33333333333333315 |  2   |
|    1553    |    230    |  0.3279411764705878 |  3   |
|    1553    |    294    | 0.26811594202898514 |  4   |
|    1553    |    125    |  0.2646153846153842 |  5   |
|    1553    |    204    | 0.24999999999999983 |  6   |
|    1553    |    248    |        0.2375       |  7   |
|    1553    |    276    | 0.23484848484848486 |  8   |
|    1553    |    155    | 0.23333333333333323 |  9   |
|    1553    |     72    | 0.22199170124481327 |  10  |
|   20400    |    226    |  0.7615894039735099 |  1   |
|   20400    |    247    | 0.33333333333333315 |  2   |
|   20400    |    230    |  0.3279411764705878 |  3   |
|   20400    |    294    | 0.26811594202898514 |  4   |
|   20400    |    125    |  0.2646153846153842 |

#### Notas
* Depois de criar o modelo, previmos os itens de recomendação usando pontuações por popularidade. Como você pode ver para cada resultado do modelo acima, as linhas mostram os primeiros 30 registros de 1000 usuários com 10 recomendações. Esses 30 registros incluem 3 usuários e seus itens recomendados, junto com a pontuação e as classificações decrescentes.
* No resultado, embora diferentes modelos tenham diferentes listas de recomendações, a cada usuário é recomendada a mesma lista de 10 itens. Isso ocorre porque a popularidade é calculada considerando os itens mais populares de todos os usuários.
* Se for um exemplo de agrupamento abaixo, os produtos 132, 248, 37 e 34 são os mais populares (mais vendidos) entre os clientes. Usando suas contagens de compras divididas pelo número de clientes, vemos que esses produtos são comprados pelo menos 3 vezes em média no conjunto de transações de treinamento (o mesmo que a primeira medida de popularidade na variável `compra_conta`)

In [39]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.321429
37     3.072797
248    3.068182
0      2.994345
34     2.955102
3      2.830544
110    2.691860
27     2.685039
245    2.678571
10     2.656442
230    2.649635
32     2.587379
226    2.542857
129    2.502732
173    2.466667
68     2.447699
41     2.398089
87     2.394309
58     2.382716
18     2.355769
Name: purchase_count, dtype: float64

## 6. Modelo de Filtragem Colaborativa

* Na filtragem colaborativa, recomendamos itens com base em como usuários semelhantes compram itens. Por exemplo, se o cliente 1 e o cliente 2 compraram itens semelhantes, por exemplo, 1 comprei X, Y, Z e 2 comprei X, Y, recomendaríamos um item Z ao cliente 2.

* Para definir a similaridade entre os usuários, usamos as seguintes etapas:
    1. Crie uma matriz de item de usuário, onde os valores de índice representam IDs de clientes exclusivos e os valores de coluna representam IDs de produtos exclusivos
    
    2. Crie uma matriz de similaridade item a item. A ideia é calcular a semelhança de um produto com outro produto. Existem várias maneiras de calcular isso. Nas etapas 6.1 e 6.2, usamos co-seno e medida de similaridade de Pearson, respectivamente.
    
        * Para calcular a similaridade entre os produtos X e Y, observe todos os clientes que avaliaram esses dois itens. Por exemplo, tanto X quanto Y foram avaliados pelos clientes 1 e 2.
        * Em seguida, criamos dois vetores de itens, v1 para o item X e v2 para o item Y, no espaço do usuário de (1, 2) e, a seguir, encontramos o ângulo / distância `cosseno` ou` pearson` entre esses vetores. Um ângulo zero ou vetores sobrepostos com valor de cosseno de 1 significam similaridade total (ou por usuário, em todos os itens, há a mesma classificação) e um ângulo de 90 graus significaria cosseno de 0 ou nenhuma similaridade.
        
    3. Para cada cliente, prevemos sua probabilidade de comprar um produto (ou suas contagens de compra) para produtos que ele não comprou.
    
        * Para o nosso exemplo, calcularemos a classificação para o usuário 2 no caso do item Z (item alvo). Para calcular isso, pesamos a medida de similaridade calculada entre o item de destino e outros itens que o cliente já comprou. O fator de pesagem são as contagens de compras fornecidas pelo usuário aos itens já adquiridos por ele.
        * Em seguida, normalizamos essa soma ponderada com a soma das medidas de similaridade para que a classificação calculada permaneça dentro dos limites predefinidos. Assim, a classificação prevista para o item Z para o usuário 2 seria calculada usando medidas de similaridade.

* Embora eu tenha escrito scripts Python para todo o processo, incluindo encontrar semelhanças usando scripts Python (que podem ser encontrados na pasta `scripts`, podemos usar a biblioteca` turicreate` por enquanto para capturar diferentes medidas, como usar distância de `cosine` e` pearson` e avalie o melhor modelo.

### 6.1. Similaridade `Cosine`
* Similaridade é o cosseno do ângulo entre os 2 vetores dos vetores dos itens de A e B
* É definido pela seguinte fórmula
![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTnRHSAx1c084UXF2wIHYwaHJLmq2qKtNk_YIv3RjHUO00xwlkt)
* Quanto mais próximos os vetores, menor será o ângulo e maior será o cosseno


#### Usando contagem de compra

In [40]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23277 users and 300 items.

Data prepared in: 0.155376s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.06591s

recommendations finished on 1000/1000 queries. users per second: 34236

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.12066556215286255  |  1   |
|    1553    |     35    | 0.07560243606567382  |  2   |
|    1553    |     1     | 0.06908496618270873  |  3   |
|    1553    |     17    | 0.057473957538604736 |  4   |
|    1553    |     33    |  0.0559393048286438  |  5   |
|    1553    |     8     | 0.04949369430541992  |  6   |
|    1553    |    105    | 0.049227023124694826 |  7   |
|    1553    |    124    | 0.04620974063873291  |  8   |
|    1553    |     5     | 0.043698668479919434 |  9   |
|    1553    |     9     | 0.037327039241790774 |  10  |
|   20400    |     1     | 0.04662901163101196  |  1   |
|   20400    |    160    | 0.043332040309906006 |  2   |
|   20400    |     31    | 0.04289144277572632  |  3   |
|   20400    |     6     | 0.04004019498825073  |  4   |
|   20400    |     77    | 0.03

#### Usando dummy

In [41]:
# these variables will change accordingly
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23299 users and 300 items.

Data prepared in: 0.15338s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.073629s

recommendations finished on 1000/1000 queries. users per second: 23602.7

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10525563359260559  |  1   |
|    1553    |     35    |  0.1003497987985611  |  2   |
|    1553    |     1     |  0.0894947499036789  |  3   |
|    1553    |     5     | 0.07986089587211609  |  4   |
|    1553    |     21    | 0.06932485103607178  |  5   |
|    1553    |     8     | 0.058724984526634216 |  6   |
|    1553    |     15    | 0.05083398520946503  |  7   |
|    1553    |     17    | 0.05068911612033844  |  8   |
|    1553    |     61    | 0.05063775181770325  |  9   |
|    1553    |     33    |  0.0492451936006546  |  10  |
|   20400    |    113    | 0.054630398750305176 |  1   |
|   20400    |     26    | 0.05153173208236694  |  2   |
|   20400    |     6     | 0.04825538396835327  |  3   |
|   20400    |    122    | 0.04676377773284912  |  4   |
|   20400    |     1     | 0.04

#### Usando contagem de compra normalizada

In [42]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23350 users and 300 items.

Data prepared in: 0.15439s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.132742s

recommendations finished on 1000/1000 queries. users per second: 56863.4

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |    224    |          0.0          |  1   |
|    1553    |    181    |          0.0          |  2   |
|    1553    |    249    |          0.0          |  3   |
|    1553    |     2     |          0.0          |  4   |
|    1553    |     10    |          0.0          |  5   |
|    1553    |    213    |          0.0          |  6   |
|    1553    |    122    |          0.0          |  7   |
|    1553    |     18    |          0.0          |  8   |
|    1553    |     72    |          0.0          |  9   |
|    1553    |    195    |          0.0          |  10  |
|   20400    |     2     |  0.003373885154724121 |  1   |
|   20400    |     1     | 0.0032870793342590333 |  2   |
|   20400    |     8     | 0.0027299237251281736 |  3   |
|   20400    |     38    |  0.002032231092453003 |  4   |
|   20400    |

### 6.2. Similaridade de `Pearson`
* Similaridade é o coeficiente de Pearson entre os dois vetores.
* É definido pela seguinte fórmula
![](http://critical-numbers.group.shef.ac.uk/glossary/images/correlationKT1.png)

#### Usando contagem de compra

In [43]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23277 users and 300 items.

Data prepared in: 0.145642s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.121311s

recommendations finished on 1000/1000 queries. users per second: 51274.2

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    132    | 3.3208039266722533 |  1   |
|    1553    |     37    | 3.0718137562503287 |  2   |
|    1553    |    248    | 3.068181818181819  |  3   |
|    1553    |     0     | 2.9943449575871828 |  4   |
|    1553    |     34    | 2.9365133071432297 |  5   |
|    1553    |     3     | 2.8305439330543947 |  6   |
|    1553    |    110    | 2.6901517668435733 |  7   |
|    1553    |     27    | 2.6850393700787416 |  8   |
|    1553    |    245    | 2.6785714285714284 |  9   |
|    1553    |     10    | 2.6564417177914104 |  10  |
|   20400    |    132    | 3.311506484236035  |  1   |
|   20400    |     37    | 3.0727969348658988 |  2   |
|   20400    |    248    | 3.068181818181819  |  3   |
|   20400    |     0     | 2.9943449575871828 |  4   |
|   20400    |     34    | 2.9551020408163255 |  5   |
|   20400 

#### Usando compra dummy

In [44]:
# these variables will change accordingly
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23299 users and 300 items.

Data prepared in: 0.161214s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.101965s

recommendations finished on 1000/1000 queries. users per second: 42443

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     23    |  0.0  |  1   |
|    1553    |     4     |  0.0  |  2   |
|    1553    |    258    |  0.0  |  3   |
|    1553    |     35    |  0.0  |  4   |
|    1553    |    173    |  0.0  |  5   |
|    1553    |     5     |  0.0  |  6   |
|    1553    |     96    |  0.0  |  7   |
|    1553    |     36    |  0.0  |  8   |
|    1553    |     19    |  0.0  |  9   |
|    1553    |     43    |  0.0  |  10  |
|   20400    |     23    |  0.0  |  1   |
|   20400    |     4     |  0.0  |  2   |
|   20400    |    258    |  0.0  |  3   |
|   20400    |     35    |  0.0  |  4   |
|   20400    |    173    |  0.0  |  5   |
|   20400    |     5     |  0.0  |  6   |
|   20400    |     96    |  0.0  |  7   |
|   20400    |     36    |  0.0  |  8   |
|   20400    |     19    |  0.0  |  9   |
|   20400    |     43    |  0.0  |  10  |
|   19750    |     23    |  0.0  |

#### Usando contagem normalizada de compra

In [45]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23350 users and 300 items.

Data prepared in: 0.154028s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.129752s

recommendations finished on 1000/1000 queries. users per second: 53538.9

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7615894039735095 |  1   |
|    1553    |    247    |  0.3333333333333333 |  2   |
|    1553    |    230    |  0.3276080699527964 |  3   |
|    1553    |    294    | 0.26811594202898537 |  4   |
|    1553    |    125    | 0.26444758561941323 |  5   |
|    1553    |    204    | 0.25000000000000006 |  6   |
|    1553    |    248    | 0.23750000000000004 |  7   |
|    1553    |    276    |  0.2347692583546495 |  8   |
|    1553    |    155    |  0.2333333333333333 |  9   |
|    1553    |     72    | 0.22199170124481324 |  10  |
|   20400    |    226    |  0.7615721508130328 |  1   |
|   20400    |    247    |  0.3333310826619466 |  2   |
|   20400    |    230    |  0.327931127127479  |  3   |
|   20400    |    294    | 0.26809407070063146 |  4   |
|   20400    |    125    |  0.2646075251469248 |

#### Observação
* Na filtragem colaborativa acima, usamos duas abordagens: cosseno e distância de Pearson. Também conseguimos aplicá-los a três conjuntos de dados de treinamento com contagens normais, dummy ou contagens normalizadas de compra de itens.
* Podemos ver que as recomendações são diferentes para cada usuário. Isso sugere que a personalização existe.
* Mas quão bom é este modelo em comparação com a linha de base e entre si? Precisamos de alguns meios de avaliar um mecanismo de recomendação. Vamos nos concentrar nisso na próxima seção.

## 7. Avaliação do modelo
Para avaliar os mecanismos de recomendação, podemos usar o conceito de recall de precisão.

* RMSE (erros quadráticos médios)
    * Mede o erro dos valores previstos
    * Quanto menor o valor RMSE, melhores as recomendações
* Lembrar
    * Qual porcentagem de produtos que um usuário compra são realmente recomendados?
    * Se um cliente comprar 5 produtos e a recomendação decidir mostrar 3 deles, o recall é de 0,6
* Precisão
    * De todos os itens recomendados, quantos o usuário realmente gostou?
    * Se 5 produtos foram recomendados ao cliente, dos quais ele compra 4 deles, a precisão é 0,8
    
* Por que o recall e a precisão são importantes?
    * Considere um caso em que recomendamos todos os produtos, para que nossos clientes com certeza tenha todos os itens que gostaria de compraram. Nesse caso, temos 100% de recall! Isso significa que nosso modelo é bom?
    * Temos que considerar a precisão. Se recomendarmos 300 itens, mas o usuário gostar e comprar apenas 3 deles, a precisão é de 0,1%! Essa precisão muito baixa indica que o modelo não é ótimo, apesar de seu excelente recall.
    * Portanto, nosso objetivo deve ser otimizar o recall e a precisão (para ser o mais próximo possível de 1).

Vamos comparar todos os modelos que construímos com base nas características de recuperação de precisão:

In [46]:
# create initial callable variables

models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

#### Modelos para contagem de compra

In [47]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13901 queries. users per second: 58548

recommendations finished on 2000/13901 queries. users per second: 53593.4

recommendations finished on 3000/13901 queries. users per second: 54962.2

recommendations finished on 4000/13901 queries. users per second: 57435.8

...

recommendations finished on 10000/13901 queries. users per second: 40440.5

recommendations finished on 11000/13901 queries. users per second: 40713.9

recommendations finished on 12000/13901 queries. users per second: 40850.4

recommendations finished on 13000/13901 queries. users per second: 39467.5


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0012229336018991443 | 0.0006634214964550903 |
|   2    | 0.0024818358391482643 |  0.002936924715933425 |
|   3    |  0.002301992662398383 | 0.0037845160510717873 |
|   4    |  0.006348464139270556 |  0.01401521463677933  |
|   5    |  0.006129055463635753 |  0.01673496661885699  |
|   6    |  0.006438385727645509 |  0.020979122857473966 |
|   7    | 0.0059913469740101016 |  0.022510761024648305 |
|   8    |  0.005629091432271062 |  0.024268171419926488 |
|   9    |  0.005275399851329696 |  0.02559609918120434  |
|   10   |  0.005244227034026349 |  0.02873701744333691  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.1244455831549338

Per User RMSE (best)
+------------+-----------------------+------

#### Modelo de compra dummy

In [48]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13928 queries. users per second: 48120.9

recommendations finished on 2000/13928 queries. users per second: 48915.3

recommendations finished on 3000/13928 queries. users per second: 50094.3

recommendations finished on 4000/13928 queries. users per second: 54698.6

...

recommendations finished on 10000/13928 queries. users per second: 52491.5

recommendations finished on 11000/13928 queries. users per second: 53651.2

recommendations finished on 12000/13928 queries. users per second: 53877.6

recommendations finished on 13000/13928 queries. users per second: 51466.8


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.007107983917288914 | 0.0034622218119489826 |
|   2    |  0.006461803561171741 |  0.006337080819590253 |
|   3    |  0.006605399195864429 |  0.009924260639897596 |
|   4    |  0.006318207926479037 |  0.01272150049558086  |
|   5    |  0.006863871338311362 |  0.017705176354472118 |
|   6    |  0.007227646946199551 |  0.022479527386361903 |
|   7    |  0.007928530401247202 |  0.029121756933890085 |
|   8    |  0.007888785180930473 |  0.033272633793553476 |
|   9    |  0.007722254132363391 |  0.03606448004641348  |
|   10   | 0.0075603101665708645 |  0.03859839728272877  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

#### Modelos de frequencia normalizado

In [49]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13815 queries. users per second: 59966.4

recommendations finished on 2000/13815 queries. users per second: 54399.6

recommendations finished on 3000/13815 queries. users per second: 55710.3

recommendations finished on 4000/13815 queries. users per second: 58283.5

...

recommendations finished on 10000/13815 queries. users per second: 41054.6

recommendations finished on 11000/13815 queries. users per second: 41432.5

recommendations finished on 12000/13815 queries. users per second: 41562.8

recommendations finished on 13000/13815 queries. users per second: 40430.2


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    |  0.002171552660151998 | 0.0012751839787670395 |
|   2    |  0.002099167571480272 |  0.002209499994516279 |
|   3    | 0.0021232959343708484 | 0.0034932681023886226 |
|   4    | 0.0020629750271444073 |  0.004604465386224347 |
|   5    |  0.002214983713355046 |  0.006274751307324599 |
|   6    | 0.0021112317529255673 |  0.007208518951189961 |
|   7    |  0.001975078848043017 |  0.007804489514587232 |
|   8    | 0.0020086862106406074 |  0.009158683299205635 |
|   9    |  0.001978525757027382 |  0.009833261529072702 |
|   10   |  0.002728917842924372 |  0.015174390621559045 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13255133365574626

Per User RMSE (best)
+------------+------------------------+----

## 8. Seleção de modelo
### 8.1. Resumo da avaliação
* Com base em RMSE


    1. Popularidade nas contagens de compra: 1,1111750034210488
    2. Similaridade de cosseno nas contagens de compra: 1,9230643981653215
    3. Similaridade de Pearson nas contagens de compra: 1,9231102838192284
    
    4. Popularidade no dummy de compra: 0,9697374361161925
    5. Similaridade de cosseno no dummy de compra: 0,9697509978436404
    6. Similaridade de Pearson no dummy de compra: 0,9697745320187097
    
    7. Popularidade em contagens de compra em normalizada: 0,16230660626840343
    8. Similaridade de cosseno em contagens de compra em normalizada: 0,16229800354111104
    9. Similaridade de Pearson nas contagens de compra em normalizada: 0,1622982668334026
    
* Com base na precisão e recall
![](../images/model_comparisons.png)


#### Notas

* Popularidade v. Filtragem colaborativa: podemos ver que os algoritmos de filtragem colaborativa funcionam melhor do que o modelo de popularidade para contagens de compras. Na verdade, o modelo de popularidade não oferece nenhuma personalização, uma vez que apenas fornece a mesma lista de itens recomendados para todos os usuários.
* Precisão e recall: Observando o resumo acima, vemos que a precisão e o recall para Contagens de compra> Dummy de compra> Contagens de compra normalizadas. No entanto, como as pontuações de recomendação para os dados de compra normalizados são zero e constantes, escolhemos a dummy. Na verdade, o RMSE não é muito diferente entre os modelos do dummy e os dos dados normalizados.
* RMSE: Como o RMSE é maior usando a distância de Pearson do que o cosseno, escolheríamos modelar os erros quadrados médios menores, que neste caso seriam o cosseno.
Portanto, selecionamos a abordagem de similaridade de cosseno no dummy de compra como nosso modelo final.

## 8. Resultado final
* Nesta etapa, gostaríamos de manipular o formato da saída de recomendação para um que possamos exportar para csv e também uma função que retornará a lista de recomendações com um ID de cliente.
* Precisamos primeiro executar novamente o modelo usando todo o conjunto de dados, pois chegamos a um modelo final usando dados de trem e avaliados com conjunto de teste.

In [50]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.160699s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.10965s

recommendations finished on 1000/1000 queries. users per second: 28884.2

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     1     | 0.10348175764083863  |  1   |
|    1553    |     2     |  0.0934672474861145  |  2   |
|    1553    |     35    |  0.0845762014389038  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.06512556076049805  |  5   |
|    1553    |     15    | 0.06476415395736694  |  6   |
|    1553    |     11    | 0.05467898845672607  |  7   |
|    1553    |     5     | 0.05406981706619263  |  8   |
|    1553    |     36    | 0.05048650503158569  |  9   |
|    1553    |     19    | 0.049363481998443606 |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

### 8.1. CSV output file

In [51]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,1,0.103482,1
1,1553,2,0.093467,2
2,1553,35,0.084576,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [52]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

#### Define função para cria a saida desejada


In [53]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('/content/option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [54]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 64366.6

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|247|230|294|125|204|248|276|155|72
11,226|247|230|294|125|204|248|276|155|213
12,226|247|230|294|125|204|248|276|155|72
16,226|247|230|294|125|204|248|276|155|72
21,226|247|230|294|125|204|248|276|155|72


### 8.2. Função de recomendação

In [55]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [56]:
customer_recomendation(4)

recommendedProducts    226|247|230|294|125|204|248|276|155|72
Name: 4, dtype: object

In [57]:
customer_recomendation(21)

recommendedProducts    226|247|230|294|125|204|248|276|155|72
Name: 21, dtype: object

## Resumo
Neste exercício, pudemos realizar um processo passo a passo para fazer recomendações aos clientes. Usamos abordagens de Filtragem Colaborativa com as medidas `cosine` e` pearson` e comparamos os modelos com nosso modelo de popularidade de linha de base. Também preparamos três conjuntos de dados que incluem contagem de compras regular, compra dummy, bem como frequência de compra normalizada como nossa variável de destino. Usando RMSE, precisão e recall, avaliamos nossos modelos e observamos o impacto da personalização. Finalmente, selecionamos a abordagem Cosine em dados de compra dummy.